In [1]:
from keras.models import Sequential
from keras.layers.core import Flatten, Dense, Dropout
from keras.layers.convolutional import Convolution2D, MaxPooling2D, ZeroPadding2D
from keras.optimizers import SGD
from keras.preprocessing import image
from keras.applications.vgg16 import preprocess_input
from keras import optimizers

import urllib
import cv2
import numpy as np

Using TensorFlow backend.


In [2]:
# Building the structure of VGGNet(with 16 layers) 
def VGG_16(weights_path=None):
    model = Sequential()
    model.add(ZeroPadding2D((1,1),input_shape=(3,224,224)))
    model.add(Convolution2D(64, 3, 3, activation='relu'))
    model.add(ZeroPadding2D((1,1)))
    model.add(Convolution2D(64, 3, 3, activation='relu'))
    model.add(MaxPooling2D((2,2), strides=(2,2), dim_ordering="th"))

    model.add(ZeroPadding2D((1,1)))
    model.add(Convolution2D(128, 3, 3, activation='relu'))
    model.add(ZeroPadding2D((1,1)))
    model.add(Convolution2D(128, 3, 3, activation='relu'))
    model.add(MaxPooling2D((2,2), strides=(2,2), dim_ordering="th"))

    model.add(ZeroPadding2D((1,1)))
    model.add(Convolution2D(256, 3, 3, activation='relu'))
    model.add(ZeroPadding2D((1,1)))
    model.add(Convolution2D(256, 3, 3, activation='relu'))
    model.add(ZeroPadding2D((1,1)))
    model.add(Convolution2D(256, 3, 3, activation='relu'))
    model.add(MaxPooling2D((2,2), strides=(2,2), dim_ordering="th"))

    model.add(ZeroPadding2D((1,1)))
    model.add(Convolution2D(512, 3, 3, activation='relu'))
    model.add(ZeroPadding2D((1,1)))
    model.add(Convolution2D(512, 3, 3, activation='relu'))
    model.add(ZeroPadding2D((1,1)))
    model.add(Convolution2D(512, 3, 3, activation='relu'))
    model.add(MaxPooling2D((2,2), strides=(2,2), dim_ordering="th"))

    model.add(ZeroPadding2D((1,1)))
    model.add(Convolution2D(512, 3, 3, activation='relu'))
    model.add(ZeroPadding2D((1,1)))
    model.add(Convolution2D(512, 3, 3, activation='relu'))
    model.add(ZeroPadding2D((1,1)))
    model.add(Convolution2D(512, 3, 3, activation='relu'))
    model.add(MaxPooling2D((2,2), strides=(2,2), dim_ordering="th"))

    model.add(Flatten())
    model.add(Dense(4096, activation='relu'))
    model.add(Dropout(0.5))
    model.add(Dense(4096, activation='relu'))
    model.add(Dropout(0.5))
    model.add(Dense(11, activation='softmax'))
    
    if weights_path:
        model.load_weights(weights_path, by_name=True)
        
    #Remove the last two layers to get the 4096D activations
    model.layers.pop()
    model.layers.pop()

    return model
    

In [3]:
# Loading the pretrained weights for VGGNet
model = VGG_16('vgg16_weights_tf_dim_ordering_tf_kernels.h5')

/home/blank/anaconda2/lib/python2.7/site-packages/keras/legacy/interfaces.py:86: UserWarning: Update your `Conv2D` call to the Keras 2 API: `Conv2D(64, (3, 3), activation="relu")`
  '` call to the Keras 2 API: ' + signature)
/home/blank/anaconda2/lib/python2.7/site-packages/keras/legacy/interfaces.py:86: UserWarning: Update your `MaxPooling2D` call to the Keras 2 API: `MaxPooling2D((2, 2), strides=(2, 2), data_format="channels_first")`
  '` call to the Keras 2 API: ' + signature)
/home/blank/anaconda2/lib/python2.7/site-packages/keras/legacy/interfaces.py:86: UserWarning: Update your `Conv2D` call to the Keras 2 API: `Conv2D(128, (3, 3), activation="relu")`
  '` call to the Keras 2 API: ' + signature)
/home/blank/anaconda2/lib/python2.7/site-packages/keras/legacy/interfaces.py:86: UserWarning: Update your `Conv2D` call to the Keras 2 API: `Conv2D(256, (3, 3), activation="relu")`
  '` call to the Keras 2 API: ' + signature)
/home/blank/anaconda2/lib/python2.7/site-packages/keras/legacy/

In [5]:
# frezzing the layers upto second dense layer so that parameters of only one dense layer are trained
for layer in model.layers:
    if layer.name == 'dense_2': break
    layer.trainable = False

In [6]:
# Compiling the model with stochastic gradient descent optimizer 
sgd = SGD(lr=0.1, decay=1e-6, momentum=0.9, nesterov=True)
# 'sparse_categorical_crossentropy' loss can be used as alternative to one hot vectors if number of classes are large
model.compile(optimizer=optimizers.SGD(lr=1e-4, momentum=0.9), 
              loss='sparse_categorical_crossentropy', metrics=['accuracy'])

In [8]:
# Loading precomputed training and validation image vectors
train_images = np.load('image_vector.npy')
valid_images = np.load('valid_image_vector.npy')

In [ ]:
# Loading precomputed training and validation label vectors
train_labels = np.load('labels_3000.npy')
valid_labels = np.load('valid_labels_500.npy')

In [ ]:
# mini_batch size for training stochastic gadient descent
batch_size = 16
# This will output trainable and non-trainable parameters of model and shapes of tensors output from each layer
model.summary()

# model.fit() returns the training accuracy, epochs define number of passes made on whole dataset
model_history = model.fit(train_images, train_labels, 
                          validation_data=(valid_images, valid_labels),
                          verbose=2,batch_size=batch_size, epochs=10)

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
zero_padding2d_1 (ZeroPaddin (None, 5, 226, 224)       0         
_________________________________________________________________
conv2d_1 (Conv2D)            (None, 3, 224, 64)        129088    
_________________________________________________________________
zero_padding2d_2 (ZeroPaddin (None, 5, 226, 64)        0         
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 3, 224, 64)        36928     
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 (None, 3, 112, 32)        0         
_________________________________________________________________
zero_padding2d_3 (ZeroPaddin (None, 5, 114, 32)        0         
_________________________________________________________________
conv2d_3 (Conv2D)            (None, 3, 112, 128)       36992     
__________